<a href="https://colab.research.google.com/github/CristianCosci/BTC_dataset_Generator_glassnode/blob/main/datasetBuilder_glassnode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests

connected = False

if(not connected):
  drive.mount('/content/drive')
  path = '/content/drive/MyDrive/progettoBTC/'
  # connected = True
  f = open("{}secret.txt".format(path))
  API_KEY = f.read().replace("\n", "")
  f.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
since = "1356998400" #get data since 01/01/2013 00:00:00
interval = "1h"

dati = []
with open(path+'t1_data.txt','r') as file:
  # reading each line
  for line in file:
      conta = 0
      # print(line)
      link = line.split()[0]
      riga = {'link': link}
      label = []
      for word in line.split()[1:]:
          label.append(word)
          conta += 1
      for i in range(0, conta, 2):
        riga[label[i]] = label[i+1]
      dati.append(riga)
      riga = {}

print(dati)

https://api.glassnode.com/v1/metrics/addresses/active_count active_addresses v

https://api.glassnode.com/v1/metrics/addresses/sending_count sending_addresses v

https://api.glassnode.com/v1/metrics/addresses/receiving_count receiving_addresses v

https://api.glassnode.com/v1/metrics/addresses/new_non_zero_count new_addresses v

https://api.glassnode.com/v1/metrics/addresses/count total_addresses v

https://api.glassnode.com/v1/metrics/market/price_usd_ohlc open o.o high o.h low o.l close o.c

https://api.glassnode.com/v1/metrics/transactions/transfers_volume_sum volume v
[{'link': 'https://api.glassnode.com/v1/metrics/addresses/active_count', 'active_addresses': 'v'}, {'link': 'https://api.glassnode.com/v1/metrics/addresses/sending_count', 'sending_addresses': 'v'}, {'link': 'https://api.glassnode.com/v1/metrics/addresses/receiving_count', 'receiving_addresses': 'v'}, {'link': 'https://api.glassnode.com/v1/metrics/addresses/new_non_zero_count', 'new_addresses': 'v'}, {'link': 'https:/

In [17]:
df = pd.DataFrame()
for i in dati:
  link = i.pop('link')
  res = requests.get(link,
  params={'a': 'BTC','s': since,'i': interval, 'api_key': API_KEY})
  foo_df = pd.json_normalize(json.loads(res.text))
  foo_df["datetime"] = pd.to_datetime(foo_df["t"], unit="s") #timestamp conversion to datetime
  foo_df = foo_df.drop("t", axis=1).set_index("datetime").sort_index()
  for j in i.keys():
    df[j] = foo_df[i[j]]

In [18]:
df

,active_addresses,sending_addresses,receiving_addresses,new_addresses,total_addresses,open,high,low,close,volume
datetime,,,,,,,,,,
2013-01-01 00:00:00,4577,2268,3801,1900,8709289,13.510010,13.510010,13.240000,13.240000,106429.243158
2013-01-01 01:00:00,1616,848,1024,647,8709936,13.240000,13.240000,13.240000,13.240000,54139.617849
2013-01-01 02:00:00,1594,856,1076,674,8710610,13.240000,13.240000,13.240000,13.240000,51321.445685
2013-01-01 03:00:00,2453,1520,1229,798,8711408,13.240000,13.240000,13.240000,13.240000,71459.679870
2013-01-01 04:00:00,2115,1064,1475,869,8712277,13.240000,13.240000,13.240000,13.240000,53602.974660
...,...,...,...,...,...,...,...,...,...,...
2022-03-29 16:00:00,59920,32968,33382,19208,958241316,47469.557984,47492.619435,47150.378261,47475.154757,104669.585370
2022-03-29 17:00:00,38633,14794,28318,15606,958256922,47492.616173,47492.616173,47229.080227,47462.534064,241418.721958
2022-03-29 18:00:00,81356,39174,52775,32145,958289067,47457.429624,47543.666124,47457.429624,47543.666124,359552.861932
